## TP3 : Méthode de réduction de variance par **``importance sampling``**

In [34]:
import numpy as np
import matplotlib.pyplot as plt

In [35]:
def simulation_brownien(T , nSamples  ) :
    """
    Simule un mouvement brownien sur l'intervalle [0, T].

    T : Durée totale de la simulation (temps final).
    nSamples :Nombre de pas de temps (échantillons) à simuler.
    G : array-like
    """
    G = np.random.randn(nSamples)
    dt = T/nSamples
    incr = np.sqrt(dt)*G 
    res = np.concatenate(([0.] , np.cumsum(incr)))
    return res 



def bsAsset(spot , r , sigma , T , nSamples , W ,  lambda_ = 0.0 ) :
    """
    Simulation de trajectoires du prix d'un actif sous-jacent dans le modèle de Black-Scholes.
    
    spot : prix initial (S0)
    r : taux sans risque
    sigma : volatilité
    T : horizon de temps
    nSamples : nombre d'échantillons 
    """
    t = np.linspace(0,T , nSamples + 1)
    res = spot*np.exp((r - 0.5*(sigma**2))*t + sigma*(W + lambda_*t))
    return res



def payoff(asset , K , L):
    return max(asset[-1] - K ,0)*int(all(asset > L))


In [36]:
def estimation_lambda(spot , r, sigma , T , nSamples , n , gamma , beta , K , L) :
    res=[0]*n 
    alpha = 1 
    for i in range(1 , n) :
        sim_brow = simulation_brownien(T , nSamples )
        # asset = bsAsset(spot , r , sigma , T , nSamples , sim_brow , res[i-1]*T)
        asset = bsAsset(spot , r , sigma , T , nSamples , sim_brow )
        lambda_ = res[i-1] - (gamma/(i + 500)**beta)*(res[i-1]*T - sim_brow[-1])*np.exp(-res[i-1]*sim_brow[-1] + (res[i-1]**2)*T/2)*(payoff(asset , K , L)**2)

        if lambda_**2 <= np.log(alpha + 1) :
            res[i]  = lambda_ 
        else : 
            res[i] = 0 
            alpha+=1
        
    return res 

In [44]:
def price(spot , r, sigma , T , nSamples , K , L , M , lambda_):

    price = 0 
    stdev = 0 
    for _ in range(M) :
        sim_brow = simulation_brownien(T , nSamples )

        asset = bsAsset(spot , r , sigma , T , nSamples , sim_brow , lambda_)
        p = payoff(asset , K , L)*np.exp(-lambda_*sim_brow[-1] - (lambda_**2)*T/2)
        price+=p
        stdev+=p**2
    
    price*=np.exp(-r*T)/M
    stdev = np.sqrt(np.exp(-2*r*T)*(stdev/M) - price**2)
    demi_ic = stdev*1.96/np.sqrt(M)
    
    return (price , demi_ic)






In [ ]:
# méthode de newton :


In [46]:
# Test :
spot = 100
K= 110
r =0.05
sigma = 0.2
T = 2
L = 80 
nSamples = 24
gamma = 0.001
beta = 0.75
n = 50000
M = 50000

lambda_ = estimation_lambda(spot , r , sigma , T , nSamples ,n , gamma ,beta , K , L )

p , ic = price(spot , r, sigma , T , nSamples , K , L , M , lambda_[-1])
print(f"pricing with importance sampling : {p} ± {ic}")
p , ic = price(spot , r, sigma , T , nSamples , K , L , M , 0)
print(f"pricing with Monte Carlo naif : {p} ± {ic}")

pricing with importance sampling : 11.207031845089952 ± 0.052273031943097535
pricing with Monte Carlo naif : 11.140988737733661 ± 0.17137099987948012
